In [ ]:
#import MEDYANSimRunner
using MEDYAN
using MEDYANVis
using StaticArrays
using LinearAlgebra
using Random
using Setfield
using OrderedCollections: OrderedDict
using SmallZarrGroups
using CairoMakie
using IterativeSolvers
using ImageBase
using Gridap
Random.seed!(1234);

In [ ]:
agent_names = MEDYAN.AgentNames(;
    diffusingspeciesnames=[
        :GA, # G-Actin with no bound nucleotide
        :GAD, # G-Actin in the ADP form
        :GADPi, # G-Actin in the ADP Pi form
        :GAT, # G-Actin in the ATP form
    ],
    filamentnames=[
        (:actin, [
            # Monomer states
            :FAD, # F-Actin in the ADP form
            :FADPi, # F-Actin in the ADP Pi form
            :FAT, # F-Actin in the ATP form
            :BarbedD, # Barbed end in the ADP form
            :BarbedDPi, # Barbed end in the ADP Pi form
            :BarbedT, # Barbed end in the ATP form
            :PointedD, # Pointed end in the ADP form
            :PointedDPi, # Pointed end in the ADP Pi form
            :PointedT, # Pointed end in the ATP form
        ])
    ]
)

In [ ]:
s = MEDYAN.SysDef(agent_names)

In [ ]:
DGActin = 5E6
for name in keys(s.diffusing)
    add_diffusion_coeff!(s, name, DGActin) # nm²/s
end

In [ ]:
add_filament_params!(s, :actin, MEDYAN.ACTIN_FIL_PARAMS)

In [ ]:
# Assume ATP, Pi, and ADP concentrations are fixed
ATP = 10000.0 * MEDYAN.μM⁻¹_per_nm³
Pi = 2000.0 * MEDYAN.μM⁻¹_per_nm³
ADP = 2000.0 * MEDYAN.μM⁻¹_per_nm³
KfGATPoint  = 0.0/MEDYAN.μM⁻¹_per_nm³
KrGATPoint  = 10.0
KfGADPoint  = 0.0/MEDYAN.μM⁻¹_per_nm³
KrGADPoint  = 10.0
KfGATBarb   = 10.0/MEDYAN.μM⁻¹_per_nm³
KrGATBarb   = 0.0
KfGADBarb   = 10.0/MEDYAN.μM⁻¹_per_nm³
KrGADBarb   = 0.0
KfPiRelease = 0.0026
KrPiRelease = 5.20E-6/MEDYAN.μM⁻¹_per_nm³

# 100
addreaction!(s,
    "diffusing.GADPi --> diffusing.GAD",
    0.0312,
    0,
)
addreaction!(s,
    "diffusing.GAD --> diffusing.GADPi",
    KrPiRelease*Pi,
    0,
)

# 104
addreaction!(s,
    "diffusing.GA --> diffusing.GAT",
    1.7/MEDYAN.μM⁻¹_per_nm³*ATP,
    0,
)
addreaction!(s,
    "diffusing.GAT --> diffusing.GA",
    0.011,
    0,
)

# 109
addreaction!(s,
    "diffusing.GA --> diffusing.GAD",
    0.9/MEDYAN.μM⁻¹_per_nm³*ADP,
    0,
)
addreaction!(s,
    "diffusing.GAD --> diffusing.GA",
    0.071,
    0,
)

In [ ]:
monomerspacing=2.7 # nm


nucleotide_states = ("T", "DPi", "D")

# Barbed end polymerization and depolymerization
barbed_rates = [
    ("T", KfGATBarb, KrGATBarb),
    ("DPi", KfGATBarb, KrGATBarb),
    ("D", KfGADBarb, KrGADBarb),
]
for (new_nuc, forward_rate, reverse_rate) in barbed_rates
    for base_nuc in nucleotide_states
        addfilamentend_reaction!(s,
            :actin,
            Symbol("f", "GA", new_nuc, "_to_Barbed", base_nuc),
            false,
            [Symbol("Barbed", base_nuc)]=>[Symbol("FA", base_nuc), Symbol("Barbed", new_nuc)],
            monomerspacing,
            "diffusing.GA"*new_nuc*" -->",
            forward_rate,
            1,
        )
        addfilamentend_reaction!(s,
            :actin,
            Symbol("r", "GA", new_nuc, "_to_Barbed", base_nuc),
            false,
            [Symbol("FA", base_nuc), Symbol("Barbed", new_nuc)]=>[Symbol("Barbed", base_nuc)],
            0.0,
            "--> diffusing.GA"*new_nuc,
            reverse_rate,
            0,
        )
    end
end

# Pointed end polymerization and depolymerization
pointed_rates = [
    ("T", KfGATPoint, KrGATPoint),
    ("DPi", KfGATPoint, KrGATPoint),
    ("D", KfGADPoint, KrGADPoint),
]
for (new_nuc, forward_rate, reverse_rate) in pointed_rates
    for base_nuc in nucleotide_states
        addfilamentend_reaction!(s,
            :actin,
            Symbol("f", "GA", new_nuc, "_to_Pointed", base_nuc),
            true,
            [Symbol("Pointed", base_nuc)]=>[Symbol("Pointed", new_nuc), Symbol("FA", base_nuc)],
            monomerspacing,
            "diffusing.GA"*new_nuc*" -->",
            forward_rate,
            1,
        )
        addfilamentend_reaction!(s,
            :actin,
            Symbol("r", "GA", new_nuc, "_to_Pointed", base_nuc),
            true,
            [Symbol("Pointed", new_nuc), Symbol("FA", base_nuc)]=>[Symbol("Pointed", base_nuc)],
            0.0,
            "--> diffusing.GA"*new_nuc,
            reverse_rate,
            0,
        )
    end
end

# Filament ATP hydrolysis and Pi Dissociation
for monomer_state in ("FA", "Barbed", "Pointed")
    addfilament_reaction!(s,
        :actin,
        Symbol(monomer_state, "T_to_", monomer_state, "DPi"),
        [Symbol(monomer_state, "T")]=>[Symbol(monomer_state, "DPi")],
        1,
        "-->",
        0.3,
        0,
    )
    addfilament_reaction!(s,
        :actin,
        Symbol(monomer_state, "DPi_to_", monomer_state, "D"),
        [Symbol(monomer_state, "DPi")]=>[Symbol(monomer_state, "D")],
        1,
        "-->",
        KfPiRelease,
        0,
    )
    addfilament_reaction!(s,
        :actin,
        Symbol(monomer_state, "D_to_", monomer_state, "DPi"),
        [Symbol(monomer_state, "D")]=>[Symbol(monomer_state, "DPi")],
        1,
        "-->",
        KrPiRelease*Pi,
        0,
    )
end
s

In [ ]:
"""
Creates an array of all monomer positions.
"""
function calc_rmon(c::MEDYAN.Context)
    rmon = SVector{3, Float64}[]
    for fil_id in filtype_fil_ids(c, 1)
        for mon_id in eachindex(fil_mon_states(c, 1, fil_id))
            name = MonomerName(1, fil_id, mon_id)
            push!(rmon,mon_position(c,name))
        end
    end
    return rmon
end

"""
Calculates the center of mass velocity of the network.
"""
function vel_com_cell(c,drmon,rmon_old,cellpervox)
    #define regular fluid grid, with cellpervox many fluid cells per 1 voxel on an edge
    fluid_grid = CubicGrid(cellpervox*c.grid.n, c.grid.compartmentsize/cellpervox)
    n_cell = MArray{Tuple{prod(fluid_grid.n)},Int64}(zeros(prod(fluid_grid.n)))
    dra_x_cell = MArray{Tuple{prod(fluid_grid.n)},Float64}(zeros(prod(fluid_grid.n)))
    dra_y_cell = MArray{Tuple{prod(fluid_grid.n)},Float64}(zeros(prod(fluid_grid.n)))
    dra_z_cell = MArray{Tuple{prod(fluid_grid.n)},Float64}(zeros(prod(fluid_grid.n)))
    #loop over all monomers accumulate array values, sort into cells using rmon_old
    for i in 1:size(rmon_old,1)
        cid = MEDYAN.grididat(fluid_grid, view(rmon_old,i)[1])
        n_cell[cid] += 1
        dra_x_cell[cid] += view(drmon,i)[1][1]
        dra_y_cell[cid] += view(drmon,i)[1][2]
        dra_z_cell[cid] += view(drmon,i)[1][3]  
    end
    va_x_cell = dra_x_cell./n_cell
    va_y_cell = dra_y_cell./n_cell
    va_z_cell = dra_z_cell./n_cell
    return va_x_cell,va_y_cell,va_z_cell
end

"""
Calculates the porosity field.
"""
function calc_por_cell(c,rmon_old,cellpervox)
    #estimate actin monomer volume by assume it is a sphere with radius 3.5 nm
    actin_rad = 3.5
    actin_vol = (4/3)*π*(actin_rad^3)
    fluid_grid = CubicGrid(cellpervox*c.grid.n, c.grid.compartmentsize/cellpervox)
    por_cell = MArray{Tuple{prod(fluid_grid.n)},Float64}(zeros(prod(fluid_grid.n)))
    #loop over all monomers accumulate array values, sort into cells using rmon_old
    for i in 1:size(rmon_old,1)
        cid = MEDYAN.grididat(fluid_grid, view(rmon_old,i)[1])
        por_cell[cid] += 1
    end
    #divide by new fluid_grid cell volume
    por_cell *= actin_vol/((fluid_grid.compartmentsize)^3)
    return por_cell
end

"""
Calculates the hydrolic permiability from porosity, using Gebart's formula.
"""
function calc_hydro_perm_cell_G(por_cell)
    #filament diameter in nm
    d = 7.0
    #fitting parameters
    ϕc = 0.0743 
    C1 = 0.491
    C2 = 2.31
    perm_cell = map(x -> (d^2)*C1*(sqrt((1-ϕc)/(1-x))-1)^C2, por_cell)
    return perm_cell
end

"""
Calculates the hydrolic permiability from porosity, using Kozeny and Carman's formula.
"""
function calc_hydro_perm_cell_KC(por_cell)
    #filament diameter in nm
    d = 7.0
    perm_cell = map(x -> inv(180)*(d^2)*(x^3)/((1-x)^2), por_cell)
    return perm_cell
end



In [ ]:
vis = Visualizer()
setvisible!(vis["/Grid"], false)
setvisible!(vis["/Axes"], false)
setvisible!(vis["/Background"], false)

In [ ]:
#system parameters
Lx = 1
Ly = 1
Lz = 3
lcell = 500
cellpervox = 2

nsec = 10
dt = 0.01
p_external = 1.0
μ = 0.28
ω = 0.1
maxiter = 40

#initialize the discrete laplacian operator
#DLO = define_3D_DLO()

#set up system and run for nsec seconds
nsubstep = Int(inv(dt))
grid = CubicGrid((Lx,Ly,Lz),lcell)
c = MEDYAN.Context(s, grid)
set_mechboundary!(c, MEDYAN.boundary_box(grid; stiffness=100.0))
adddiffusingcount_rand!(c, s.diffusing.GAT, 700)
monomerstates = [s.state.actin.PointedD, s.state.actin.FADPi, s.state.actin.FAT, s.state.actin.BarbedT]
nodepositions = [SA[0.0,0.0,0.0], SA[0.0,0.0,monomerspacing*length(monomerstates)]]
chem_newfilament!(c; monomerstates, nodepositions, node_mids=[1,])

for i in 1:nsec
    for j in 1:nsubstep
        run_chemistry!(c,dt)
        
        #get n-1-th minimized state
        rmon_old = calc_rmon(c)
        
        #get n-th minimized state
        minimize_energy!(c)
        rmon_new = calc_rmon(c)
        drmon = rmon_new .- rmon_old
        
        #calculate the network velocity
        va_x_cell, va_y_cell, va_z_cell = vel_com_cell(c,drmon,rmon_old,cellpervox)
        #calculate the porosity field
        por_cell = calc_por_cell(c,rmon_old,cellpervox)
        #calculate the hydrolic permiability tensor element
        perm_cell = calc_hydro_perm_cell_KC(por_cell)
        
        #calculate the poisson equation source term
        

        draw_context!(vis, c, s)
    end
end